# Viability tube for quadrotor

**Adapted from**: [YAP21, Section V.D] for the model defined in [B12], [M16, Section IV] and [M19, Section 6.1]

[B12] Bouffard, Patrick.
*On-board model predictive control of a quadrotor helicopter: Design, implementation, and experiments.*
CALIFORNIA UNIV BERKELEY DEPT OF COMPUTER SCIENCES, 2012.

[M16] Mitchell, Ian M., et al.
*Ensuring safety for sampled data systems: An efficient algorithm for filtering potentially unsafe input signals.*
2016 IEEE 55th Conference on Decision and Control (CDC). IEEE, 2016.

[M19] Mitchell, Ian M., Jacob Budzis, and Andriy Bolyachevets.
*Invariant, viability and discriminating kernel under-approximation via zonotope scaling.*
Proceedings of the 22nd ACM International Conference on Hybrid Systems: Computation and Control. 2019.

[YAP21] Yin, H., Arcak, M., Packard, A., & Seiler, P. (2021).
*Backward reachability for polynomial systems on a finite horizon.*
IEEE Transactions on Automatic Control, 66(12), 6025-6032.

In [1]:
using DynamicPolynomials
@polyvar x[1:6]
@polyvar u[1:2]
sinx5 = -0.166 * x[5]^3 + x[5]
cosx5 = -0.498 * x[5]^2 + 1
gn = 9.8
K = 0.89 / 1.4
d0 = 70
d1 = 17
n0 = 55
f = [
    x[3],
    x[4],
    0,
    -gn,
    x[6],
    -d0 * x[5] - d1 * x[6],
]
n_x = length(f)
g = [
    0         0
    0         0
    K * sinx5 0
    K * cosx5 0
    0         0
    0         n0
]
n_u = size(g, 2)

2

The constraints below are the same as [YAP21, M16, M19] except
[M16, M19] uses different bounds for `x[2]` and
[M16] uses different bounds for `x[5]`

In [2]:
using SumOfSquares
rectangle = [1.7, 0.85, 0.8, 1, π/12, π/2, 1.5, π/12]
X = BasicSemialgebraicSet(FullSpace(), typeof(x[1] + 1.0)[])
for i in eachindex(x)
    addinequality!(X, x[i] + rectangle[i]) # x[i] >= -rectangle[i]
    addinequality!(X, rectangle[i] - x[i]) # x[i] <= rectangle[i]
end
X

Basic semialgebraic Set defined by no equality
12 inequalities
 x[1] + 1.7 ≥ 0
 -x[1] + 1.7 ≥ 0
 x[2] + 0.85 ≥ 0
 -x[2] + 0.85 ≥ 0
 x[3] + 0.8 ≥ 0
 -x[3] + 0.8 ≥ 0
 x[4] + 1.0 ≥ 0
 -x[4] + 1.0 ≥ 0
 x[5] + 0.2617993877991494 ≥ 0
 -x[5] + 0.2617993877991494 ≥ 0
 x[6] + 1.5707963267948966 ≥ 0
 -x[6] + 1.5707963267948966 ≥ 0


The starting value for `k` is the following linear state-feedback
that maintains the quadrotor at the origin [YAP21, Remark 3].

For this, we compute an ellipsoidal control invariant set using [LJ21, Corollary 9]
For this, we first compute the descriptor system described in [LJ21, Proposition 5].

[LJ21] Legat, Benoît, and Jungers, Raphaël M.
*Geometric control of algebraic systems.*
IFAC-PapersOnLine 54.5 (2021): 79-84.

In [3]:
using SparseArrays
x0 = zeros(n_x)
u0 = [gn/K, 0.0]

2-element Vector{Float64}:
 15.415730337078651
  0.0

The linearization of `f` is given by

In [4]:
A = map(differentiate(f, x)) do f
    f(x => x0)
end

6×6 Matrix{Float64}:
 0.0  0.0  1.0  0.0    0.0    0.0
 0.0  0.0  0.0  1.0    0.0    0.0
 0.0  0.0  0.0  0.0    0.0    0.0
 0.0  0.0  0.0  0.0    0.0    0.0
 0.0  0.0  0.0  0.0    0.0    1.0
 0.0  0.0  0.0  0.0  -70.0  -17.0

The linearization of `g` is given by:

In [5]:
B = map(g) do g
    g(x => x0)
end

6×2 Matrix{Float64}:
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.635714   0.0
 0.0        0.0
 0.0       55.0

We can see that the equilibrium is not stabilizable.
Indeed, if `x3` is nonzero then `x1` will grow indefinitely
while we have no control over either `x1` nor `x3`.
Let's find a Lyapunov function for the rest of the states
and compute a linear state feedback for these.

In [6]:
J = setdiff(1:n_x, [1, 3])
nJ = length(J)
nD = nJ + n_u
E = sparse(1:nJ, 1:nJ, ones(nJ), nJ, nD)
AJ = A[J, J]
BJ = B[J, :]
C = [AJ BJ]

4×6 Matrix{Float64}:
 0.0  1.0    0.0    0.0  0.0        0.0
 0.0  0.0    0.0    0.0  0.635714   0.0
 0.0  0.0    0.0    1.0  0.0        0.0
 0.0  0.0  -70.0  -17.0  0.0       55.0

We know solve [LJ21, (13)]

In [7]:
using LinearAlgebra
import SCS
solver = optimizer_with_attributes(SCS.Optimizer, MOI.Silent() => true)
model = Model(solver)
@variable(model, Q[1:nD, 1:nD] in PSDCone())
cref = @constraint(model, Symmetric(-C * Q * E' - E * Q * C') in PSDCone())
rectangle_J = rectangle[[J; nJ .+ (1:n_u)]]
@constraint(model, rect_ref[i in 1:nD], Q[i, i] <= rectangle[i])
@variable(model, volume)
q = [Q[i, j] for j in 1:nD for i in 1:j]
@constraint(model, [volume; q] in MOI.RootDetConeTriangle(nD))
@objective(model, Max, volume)
optimize!(model)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 6.24704e-01
  Dual objective value : 6.24693e-01

* Work counters
  Solve time (sec)   : 6.10355e-02


We now have the control-Lyapunov function `V(x, u) = [x, u]' inv(Q) [x, u]`.
In other words, The 1-sublevel set of `V(x, u)` is an invariant subset of `rectangle`
with any state-feedback `κ(x)` such that `V(x, κ(x)) ≤ 1` for any `x` such that
`min_u V(x, u) ≤ 1`.
Such candidate `κ(x)` can therefore be chosen as `argmin_u V(x, u)`.
Let `inv(Q) = U' * U` where `U = [Ux Uu]`. We have `V(x, u) = ||Ux * x + Uu * u||_2`.
`κ(x)` is therefore the least-square solution of `Uu * κ(x) = -Ux * x`.
This we find the linear state-feedback `κ(x) = K * x` where `K = -Uu \ Ux`.

In [8]:
P = inv(Symmetric(value.(Q)))
using LinearAlgebra
F = cholesky(P)
K = -F.U[:, (nJ + 1):(nD)] \ F.U[:, 1:nJ] # That gives the following state feedback in polynomial form:

k = K * x[J]

2-element Vector{DynamicPolynomials.Polynomial{true, Float64}}:
 -0.19622653627237438x₂ - 0.39287266922367087x₄ - 3.930388471170766e-16x₅ - 2.5763934158713317e-16x₆
 2.6828640723569464e-15x₂ - 6.523042058279634e-15x₄ + 0.9534429382637486x₅ - 0.11396000421436397x₆

We now have two equivalent ways to obtain the Lyapunov function.
Because `{V(x) ≤ 1} = {min_u V(x, u) ≤ 1}`,
see the left-hand side as the projection of the ellipsoid on `x, u`.
As the projection on the polar becomes simply cutting with the hyperplane `u = 0`,
the polar of the projection is simply `Q[1:6, 1:6]` ! So

In [9]:
Px = inv(Symmetric(value.(Q[1:nJ, 1:nJ])))

4×4 LinearAlgebra.Symmetric{Float64, Matrix{Float64}}:
 0.661753      0.441601      2.86668e-15   2.29212e-15
 0.441601      2.65257      -5.85708e-15  -1.691e-15
 2.86668e-15  -5.85708e-15   1.69023       0.449148
 2.29212e-15  -1.691e-15     0.449148      1.11935

An alternative way is to use our linear state feedback.
We know that `min_u V(x, u) = V(x, Kx)` so

In [10]:
Px = [I; K]' * P * [I; K]

4×4 Matrix{Float64}:
 0.661753      0.441601      2.86668e-15   2.29212e-15
 0.441601      2.65257      -5.85708e-15  -1.691e-15
 2.86668e-15  -5.85708e-15   1.69023       0.449148
 2.29212e-15  -1.691e-15     0.449148      1.11935

We can double check that this matrix is negative definite:

In [11]:
eigen(Symmetric(Px * (AJ + BJ * K) + (AJ + BJ * K)' * Px)).values

4-element Vector{Float64}:
 -66.96611787243293
  -0.5519161080356569
  -4.058120947211065e-5
  -1.1635183326520363e-5

Let's now find a valid Lyapunov function for the nonlinear system
using that linear state feedback.
That corresponds to the V-step of [YAP21, Algorithm 1]:

In [12]:
function _create(model, d, P)
    if d isa Int
        return @variable(model, variable_type = P(monomials(x, 0:d)))
        #return @variable(model, variable_type = P(monomials([t; x], 0:d)))
    else
        return d
    end
end
function base_model(solver, V, k, s3, γ)
    model = SOSModel(solver)
    V = _create(model, V, Poly)
    k = _create.(model, k, Poly)
    s3 = _create(model, s3, SOSPoly)
    ∂ = differentiate # Let's use a fancy shortcut
    @constraint(model, ∂(V, x) ⋅ (f + g * k) <= s3 * (V - γ)) # [YAP21, (E.2)]
    for r in inequalities(X)
        @constraint(model, V >= γ, domain = @set(r >= 0)) # [YAP21, (E.3)]
    end
    return model, V, k, s3
end

_degree(d::Int) = d
_degree(V) = maxdegree(V)

function V_step(solver, V0, γ, k, s3)
    model, V, k, s3 = base_model(solver, _degree(V0), k, s3, γ)
    if !(V0 isa Int)
        @constraint(model, V >= γ, domain = @set(V0 >= γ)) # [YAP21, (E.6)]
    end
    optimize!(model)
    return model, value(V)
end

γ = 1.0
s3 = 1.0
model, V = V_step(solver, 2, γ, k, s3)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : -7.48602e-12

* Work counters
  Solve time (sec)   : 6.83004e-03


The Lyapunov obtained is as follows

In [13]:
V

3.270918395732398e-8x₁² + 3.167809298034239e-23x₁x₂ - 2.530765421021385e-12x₁x₃ - 1.6957568763743917e-24x₁x₄ - 4.046968102637713e-12x₁x₅ + 7.155764089773019e-13x₁x₆ + 1.3704656436223044e-7x₂² - 1.700600017936993e-23x₂x₃ - 1.745266763178922e-7x₂x₄ - 6.349598762228351e-22x₂x₅ + 3.751457724049311e-22x₂x₆ + 6.917050716923042e-12x₃² + 9.069404342588527e-25x₃x₄ + 1.4704199201121313e-13x₃x₅ - 2.3070604887078734e-15x₃x₆ + 1.5612394153649966e-7x₄² + 3.8838925942527625e-21x₄x₅ + 2.0809806260551552e-22x₄x₆ + 3.631738895961291e-7x₅² - 1.2963928636526255e-9x₅x₆ + 7.072583336286186e-8x₆² - 2.906254993885284e-21x₁ + 5.081950998123909e-7x₂ - 8.725723805070821e-22x₃ - 4.1379474225663187e-7x₄ - 1.2636023067916625e-20x₅ - 1.9387032398637226e-21x₆ + 1.0000068335029546

We now try to find a state feedback that would improve γ

In [14]:
using MutableArithmetics
function γ_step(solver, V, γ_min, k_best, s3_best, degree_k, degree_s3, γ_tol, max_iters)
    γ0_min = γ_min
    γ_max = Inf
    num_iters = 0
    while γ_max - γ_min > γ_tol && num_iters < max_iters
        if isfinite(γ_max)
            γ = (γ_min + γ_max) / 2
        else
            γ = γ0_min + (γ_min - γ0_min + 1) * 2
        end
        model, V, k, s3 = base_model(solver, V, degree_k, degree_s3, γ)
        num_iters += 1
        @info("Iteration $num_iters/$max_iters : solving...")
        optimize!(model)
        if primal_status(model) == MOI.FEASIBLE_POINT
            γ_min = γ
            k_best = value.(k)
            s3_best = value(s3)
        elseif dual_status(model) == MOI.INFEASIBILITY_CERTIFICATE
            γ_max = γ
        else
            @warn("Giving up $(raw_status(model)), $(termination_status(model)), $(primal_status(model)), $(dual_status(model))")
            break
        end
        @info("Solved in $(solve_time(model)) : γ ∈ ]$γ_min, $γ_max]")
    end
    if !isfinite(γ_max)
        error("Cannot find any infeasible γ")
    end
    return γ_min, k_best, s3_best
end

γ, k, s3 = γ_step(solver, V, γ, k, s3, [2, 2], 2, 1e-3, 10)

[ Info: Iteration 1/10 : solving...
[ Info: Solved in 0.092617425 : γ ∈ ]1.0, 3.0]
[ Info: Iteration 2/10 : solving...
[ Info: Solved in 0.09151751899999999 : γ ∈ ]1.0, 2.0]
[ Info: Iteration 3/10 : solving...
[ Info: Solved in 0.087750497 : γ ∈ ]1.0, 1.5]
[ Info: Iteration 4/10 : solving...
[ Info: Solved in 0.088683104 : γ ∈ ]1.0, 1.25]
[ Info: Iteration 5/10 : solving...
[ Info: Solved in 0.11490145199999999 : γ ∈ ]1.0, 1.125]
[ Info: Iteration 6/10 : solving...
[ Info: Solved in 0.086079088 : γ ∈ ]1.0, 1.0625]
[ Info: Iteration 7/10 : solving...
[ Info: Solved in 0.087819998 : γ ∈ ]1.0, 1.03125]
[ Info: Iteration 8/10 : solving...
[ Info: Solved in 0.08938420700000001 : γ ∈ ]1.0, 1.015625]
[ Info: Iteration 9/10 : solving...
[ Info: Solved in 0.08630818999999999 : γ ∈ ]1.0, 1.0078125]
[ Info: Iteration 10/10 : solving...
[ Info: Solved in 0.08896680500000001 : γ ∈ ]1.0, 1.00390625]


(1.0, DynamicPolynomials.Polynomial{true, Float64}[-0.19622653627237438x₂ - 0.39287266922367087x₄ - 3.930388471170766e-16x₅ - 2.5763934158713317e-16x₆, 2.6828640723569464e-15x₂ - 6.523042058279634e-15x₄ + 0.9534429382637486x₅ - 0.11396000421436397x₆], 1.0)

We now try to find a new Lyapunov V:

In [15]:
model, V = V_step(solver, V, γ, k, s3)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : -7.83079e-12

* Work counters
  Solve time (sec)   : 7.68374e-03


The Lyapunov obtained is as follows

In [16]:
V

2.6887299233504928e-8x₁² + 2.9872456866778983e-22x₁x₂ - 2.1001715283002107e-12x₁x₃ - 1.0564783856845026e-23x₁x₄ - 4.1701741482069905e-12x₁x₅ + 6.081130476278392e-13x₁x₆ + 1.3292427676820004e-7x₂² - 1.2455396604481166e-24x₂x₃ - 1.652986627998994e-7x₂x₄ + 9.704285178591127e-22x₂x₅ + 3.110818506251064e-22x₂x₆ + 5.689477455633206e-12x₃² + 4.0087452607583566e-24x₃x₄ - 3.7082957182125265e-13x₃x₅ + 1.936766477769727e-15x₃x₆ + 1.474520495638035e-7x₄² + 2.105424731073417e-21x₄x₅ + 3.309792931490456e-22x₄x₆ + 3.252095639402081e-7x₅² - 8.895436954412837e-10x₅x₆ + 6.573719564305073e-8x₆² - 1.9316086180918378e-21x₁ + 4.7073526459043376e-7x₂ - 3.422343936884572e-21x₃ - 3.924799482096176e-7x₄ - 5.7908724195359595e-21x₅ - 1.6851565478676564e-21x₆ + 1.000006466375888

We now try to improve γ again

In [17]:
γ, k, s3 = γ_step(solver, V, γ, k, s3, [2, 2], 2, 1e-3, 10)

[ Info: Iteration 1/10 : solving...
[ Info: Solved in 0.09087311599999999 : γ ∈ ]1.0, 3.0]
[ Info: Iteration 2/10 : solving...
[ Info: Solved in 0.09133789899999999 : γ ∈ ]1.0, 2.0]
[ Info: Iteration 3/10 : solving...
[ Info: Solved in 0.08670916599999999 : γ ∈ ]1.0, 1.5]
[ Info: Iteration 4/10 : solving...
[ Info: Solved in 0.08758816999999999 : γ ∈ ]1.0, 1.25]
[ Info: Iteration 5/10 : solving...
[ Info: Solved in 0.090765587 : γ ∈ ]1.0, 1.125]
[ Info: Iteration 6/10 : solving...
[ Info: Solved in 0.087583569 : γ ∈ ]1.0, 1.0625]
[ Info: Iteration 7/10 : solving...
[ Info: Solved in 0.086625065 : γ ∈ ]1.0, 1.03125]
[ Info: Iteration 8/10 : solving...
[ Info: Solved in 0.087804372 : γ ∈ ]1.0, 1.015625]
[ Info: Iteration 9/10 : solving...
[ Info: Solved in 0.08982508200000001 : γ ∈ ]1.0, 1.0078125]
[ Info: Iteration 10/10 : solving...
[ Info: Solved in 0.088008373 : γ ∈ ]1.0, 1.00390625]


(1.0, DynamicPolynomials.Polynomial{true, Float64}[-0.19622653627237438x₂ - 0.39287266922367087x₄ - 3.930388471170766e-16x₅ - 2.5763934158713317e-16x₆, 2.6828640723569464e-15x₂ - 6.523042058279634e-15x₄ + 0.9534429382637486x₅ - 0.11396000421436397x₆], 1.0)

We now try to find a new Lyapunov V:

In [18]:
model, V = V_step(solver, V, γ, k, s3)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : -7.83083e-12

* Work counters
  Solve time (sec)   : 7.12984e-03


The Lyapunov obtained is as follows

In [19]:
V

2.6887299283227056e-8x₁² + 2.8762797147229776e-22x₁x₂ - 2.100171534793294e-12x₁x₃ + 7.476442203909692e-25x₁x₄ - 4.170174149078067e-12x₁x₅ + 6.081130486979986e-13x₁x₆ + 1.3292427680563458e-7x₂² + 1.9358083615283524e-23x₂x₃ - 1.652986627604508e-7x₂x₄ + 7.647947358637265e-22x₂x₅ + 3.873503353002841e-22x₂x₆ + 5.689477467238331e-12x₃² - 9.861014506118508e-25x₃x₄ - 3.7082956913559794e-13x₃x₅ + 1.9367664554255644e-15x₃x₆ + 1.474520495340144e-7x₄² + 2.7897772730640074e-21x₄x₅ + 2.1266127707066577e-22x₄x₆ + 3.252095640491712e-7x₅² - 8.895436795628335e-10x₅x₆ + 6.573719564113916e-8x₆² + 6.0072488023391475e-22x₁ + 4.7073526494024047e-7x₂ + 1.0450303630378034e-21x₃ - 3.9247994881491304e-7x₄ - 8.33211884396278e-21x₅ - 1.834370496240584e-21x₆ + 1.0000064663758899

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*